Сухоруков Кирилл Евгеньевич

сайт - https://memepedia.ru/

парсить изображение И текст

# Импорт библиотек

In [1]:
%pip install requests
%pip install bs4

In [2]:
import pandas as pd
import requests
import re
import bs4
import os
import datetime as dtime
from bs4 import BeautifulSoup
from threading import Thread

# W/o threading: 2533.770363330841s  |   42,229506055514m
# W/ threading:  695.3645355701447s  |   11,5894089261691m
# Скорость работы отличается в 3,6438 раз.

# Подключение Google Drive

In [3]:
def makedir(path: str):
  """Создание папки

  Parameters
  :param path - Путь создаваемой папки
  :type path: str
  """
  try:
    os.makedirs(path, exist_ok=True)
  except Exception as e:
    print('Error: ', e)

In [4]:
from google.colab import drive
drive.mount('/content/drive')
work_dir='/content/drive/My Drive/Colab Notebooks/'
memes_folder = work_dir + 'Memes_Folder/'
makedir(memes_folder)

Mounted at /content/drive


# Парсинг сайта

In [5]:
def scrape_page(url: str):
    """Собрать верстку страницы
    
    Parameters
    :param url - Ссылка на страницу
    :type url: str

    Return
    :return soup - Полученная верстка страницы
    :type soup: bs4.BeautifulSoup
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    return soup

In [6]:
def get_publication_time(soup: bs4.BeautifulSoup):
    """Получить время публикации статьи

    Parameters
    :param soup - Код страницы статьи 
    :type soup: bs4.BeautifulSoup

    Return
    :return publication_time
    :type publication_time: str
    """
    publication_time = soup.find('span', class_="auth-posted-on").\
                      contents[0]['datetime']
    return publication_time

In [7]:
def get_view_count(soup: bs4.BeautifulSoup):
    """Получить число просмотров статьи
      
    Parameters
    :param soup - Код страницы статьи 
    :type soup: bs4.BeautifulSoup

    Return
    :return view_count
    :type view_count: str

    """
    view_count = soup.find('div', class_='s-post-main').\
                      find('span', class_='count').text
    view_count = float(view_count[:-1]) * 1000
    return view_count

In [8]:
def get_post_description(soup: bs4.BeautifulSoup):
    """Получить описание мема

    Parameters
    :param soup - Код страницы статьи 
    :type soup: bs4.BeautifulSoup

    Return 
    :return description - Описание мема
    :type description: str
    """
    description = soup.find('div', itemprop = 'articleBody').find('p').text
    return description

In [9]:
def get_post_category_tags(post: bs4.element.Tag):
    """Получить теги категорий поста

    Parameters
    :param post
    :type post: bs4.element.Tag

    Return
    :return category_tags
    :type category_tags: str
    """
    tags_marks = post.header.find_all('a', rel="category tag")
    category_tags = ", ".join([i.text for i in tags_marks])
    return category_tags

In [27]:
def get_post_content_tags(soup: bs4.BeautifulSoup):
    """Получить теги содержания поста

    Parameters
    :param soup - Код страницы статьи 
    :type soup: bs4.BeautifulSoup

    Return 
    :return content_tags - Теги содержания мема
    :type content_tags: str
    """
    content_tags_marks = soup.find_all('meta', property="article:tag")
    #Так как теги содержания поста дублируются в коде страницы,
    # берем только половину полученного списка
    content_tags_marks = content_tags_marks[:(len(content_tags_marks)//2)] 
    content_tags = ', '.join([i['content'] for i in content_tags_marks])
    return content_tags

In [11]:
def get_image(soup: bs4.BeautifulSoup):
    """Получить изображение мема

    Parameters
    :param soup - Код страницы статьи
    :type soup: bs4.BeautifulSoup

    Return
    :return img_bytes - Изображение мема, полученное от сервера
    :type img_bytes: requests.models.Response
    """
    thumbnail = soup.find("figure", class_="post-thumbnail")
    if thumbnail.find("img"):
        link = thumbnail.find("img")["src"]
    else:
        link = thumbnail.find("video")["poster"]
    img_bytes = requests.get(link)
    return link, img_bytes

In [31]:
def scrape_meme(post: bs4.element.Tag, post_info: list, index: int = 0):
    """Скачать информацию о меме и записать текстовые данные в указанный список

    Parameters
    :param post - Обрабатываемый мем
    :type post: bs4.element.Tag

    :param post_info - Список, в который будут записаны текстовые данные мема
    :type post_info: list

    :param index - Индекс потока
    :type index: int (default 0)

    Return
    :return True - сигнал об успешном завершении потока
    :type True: bool
    """
    regex = re.compile(r'[\\/:\*\?"<>|]')

    post_link = post.find("a")["href"]
    post_soup = scrape_page(post_link)

    name = post.find("a")["title"]
    if re.search(regex, name):
        filename = regex.sub(" - ", name) + ".jpg"
    else:
        filename = name + ".jpg"
    file_link = ''
    category_tags = get_post_category_tags(post)
    content_tags = get_post_content_tags(post_soup)
    publication_time = dtime.datetime.strptime(
        get_publication_time(post_soup), "%Y-%m-%dT%H:%M:%S%z"
    )
    view_count = get_view_count(post_soup)
    meme_description = get_post_description(post_soup)

    year_folder = memes_folder + str(publication_time.year) + "/"
    makedir(year_folder)
    with open(year_folder + filename, "wb") as f:
        img_bytes = get_image(post_soup)
        file_link = img_bytes[0]
        f.write(img_bytes[1].content)

    df_entry = {
        "Meme Name": name,
        "File Name": filename,
        "File Link": file_link,
        "Publication Time": publication_time,
        'View Count': view_count,
        "Category Tags": category_tags,
        "Content Tags": content_tags,
        "Meme Description": meme_description,
    }

    post_info[index] = df_entry
    return True

In [13]:
def scrape_posts(posts: bs4.element.ResultSet):
    """Скачать все посты со страницы и вернуть список текстовых
    данных этих постов

      Parameters
      :param posts - Набор постов со страницы
      :type posts: bs4.element.ResultSet

      Return
      :return posts_info - Список данных по каждому посту
      :type posts_info: list
    """
    posts_info = [{} for x in posts]
    threads = []
    for ii in range(len(posts)):
        process = Thread(target=scrape_meme, args=[posts[ii], posts_info, ii])
        process.start()
        threads.append(process)

    for process in threads:
        process.join()

    return posts_info

In [35]:
memoteka_url = "https://memepedia.ru/memoteka/"
soup = scrape_page(memoteka_url)
page_count_links = soup.find_all("a", class_="page-numbers")
# pages = range(int(page_count_links[-2].text))
pages = range(1)
df = pd.DataFrame(
    columns=[
        "Meme Name",
        "File Name",
        "File Link",
        "Publication Time",
        "View Count",
        "Category Tags",
        "Content Tags", 
        "Meme Description",
    ]
)

In [ ]:
ss = scrape_page("https://memepedia.ru/naskolko-zhe-ty-otchayalsya/")
get_post_content_tags(ss)

In [ ]:
for page in pages:
    page_str = f"page/{page+1}/"
    page_soup = scrape_page(memoteka_url + page_str)
    print(page + 1)
    posts = page_soup.find_all("article", class_="bb-card-item")
    posts_data = scrape_posts(posts)
    df = df.append(posts_data)

In [41]:
meme_dbase_filename = "Meme_DBase.csv"
df.reset_index(inplace=True)

In [ ]:
df

In [ ]:
print(df)
df.to_csv(
    memes_folder + meme_dbase_filename,
    index=False,
)
df.head()